# Intent Classification

## Importing The Libraries

In [1]:
import numpy as np
import pickle
import random
import json

import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

## Preparing & Preprocessing Dataset

In [2]:
words=[]
classes = []
documents = []

ignore_words = ['?','!']

data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intent']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
print("_" * 80)

# classes = intents
print ("Num of Intents",len(classes), "classes",'\n',classes)
print("_" * 80)

# words =  vocabulary
print (len(words), "unique lemmatized words", '\n',words)

232 documents
________________________________________________________________________________
Num of Intents 13 classes 
 ['Alarm', 'Cooking', 'Date', 'Goodbye', 'Greeting', 'Info_Request', 'Iot', 'Joke', 'PlayMusic', 'Thanks', 'Time', 'Transport', 'Weather']
________________________________________________________________________________
358 unique lemmatized words 
 ["'s", ',', '.', '10', '4', '[', ']', 'a', 'about', 'according', 'active', 'afternoon', 'alarm', 'alert', 'am', 'an', 'any', 'anyone', 'april', 'are', 'around', 'artist', 'ask', 'at', 'awesome', 'bad', 'bathroom', 'be', 'bedroom', 'before', 'berlin', 'best', 'better', 'billie', 'bit', 'blue', 'boil', 'book', 'boston', 'bowie', 'bread', 'bridge', 'brief', 'brightener', 'brighter', 'brightness', 'build', 'business', 'by', 'bye', 'cairo', 'can', 'cancel', 'carpet', 'change', 'cheescake', 'cheese', 'clean', 'cleaner', 'cleaning', 'cold', 'coldplay', 'come', 'comic', 'concert', 'cook', 'cookie', 'cooking', 'cornwall', 'could'

## Create Training Data

In [3]:
# create our training data
train = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    train.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(train)
train = np.array(train,dtype=object)

# create train and test lists. X - patterns, Y - intents
train_x = list(train[:,0])
train_y = list(train[:,1])
print("Training data created")

Training data created


## Model Preparation & Training

In [4]:
# Create model - 3 layers. 
# 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [5]:
# Compile model. Stochastic gradient descent 
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [6]:
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
47/47 [==============================] - 0s 867us/step - loss: 2.4907 - accuracy: 0.1638
Epoch 2/200
47/47 [==============================] - 0s 898us/step - loss: 2.1983 - accuracy: 0.2500
Epoch 3/200
47/47 [==============================] - 0s 835us/step - loss: 1.8955 - accuracy: 0.4052
Epoch 4/200
47/47 [==============================] - 0s 869us/step - loss: 1.4752 - accuracy: 0.5517
Epoch 5/200
47/47 [==============================] - 0s 876us/step - loss: 1.2849 - accuracy: 0.6336
Epoch 6/200
47/47 [==============================] - 0s 864us/step - loss: 1.1183 - accuracy: 0.6509
Epoch 7/200
47/47 [==============================] - 0s 650us/step - loss: 0.8228 - accuracy: 0.7629
Epoch 8/200
47/47 [==============================] - 0s 658us/step - loss: 0.7812 - accuracy: 0.7629
Epoch 9/200
47/47 [==============================] - 0s 650us/step - loss: 0.6808 - accuracy: 0.7888
Epoch 10/200
47/47 [==============================] - 0s 880us/step - loss: 0.5658 - accura

Epoch 161/200
47/47 [==============================] - 0s 875us/step - loss: 0.0221 - accuracy: 0.9957
Epoch 162/200
47/47 [==============================] - 0s 979us/step - loss: 0.0172 - accuracy: 0.9957
Epoch 163/200
47/47 [==============================] - 0s 754us/step - loss: 0.0064 - accuracy: 1.0000
Epoch 164/200
47/47 [==============================] - 0s 650us/step - loss: 0.0058 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 875us/step - loss: 0.0106 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 872us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 650us/step - loss: 0.0268 - accuracy: 0.9914
Epoch 168/200
47/47 [==============================] - 0s 867us/step - loss: 0.0112 - accuracy: 0.9957
Epoch 169/200
47/47 [==============================] - 0s 654us/step - loss: 0.0044 - accuracy: 1.0000
Epoch 170/200
47/47 [==============================] - 0s 650us/step - lo

## Saving Model, Classes and Words

In [7]:
model.save('Model.h5',history)

with open('classes.pkl','wb') as file:  
    pickle.dump(classes,file)

with open('words.pkl','wb') as file:  
    pickle.dump(words,file)

--------------------------------------------------------------------------------------------------------


# Name Entity Recognition

## Importing The Libraries

In [8]:
from __future__ import unicode_literals, print_function
import plac
from pathlib import Path
from spacy.util import minibatch , compounding
from spacy.training import Example

## Train Data

In [9]:
data_file = open('entities.json').read()


entities = json.loads(data_file)
train_data = []


for entity in entities['train_data']:
    train_data.append(entity)

## Append training data

def built_ner(text, target, type):
    start = str.find(text, target)
    end = start + len(target)
    
    return (text, {"entities": [(start, end, type)]})

#train_data.append(built_ner(text,target,type))
train_data.append(built_ner("I want to create a cloud-based service connected to Revit Server.", "Revit Server", "PRODUCT"))
train_data.append(built_ner("I'm new to the Forge API unsure where a design parameter is required", "Forge API", "API"))
train_data.append(built_ner("Autodesk Forge is my Platform of choice", "Autodesk Forge", "PRODUCT"))
train_data.append(built_ner("I've uploaded a Revit model to my OSS bucket.", "OSS", "SERVICE"))
train_data.append(built_ner("Changes are sent to a central BIM 360 server.", "BIM 360", "PRODUCT"))
train_data.append(built_ner("All of this is possible on the IFC format.", "IFC", "FORMAT"))
train_data.append(built_ner("The native file format for Revit is RVT.", "RVT", "FORMAT"))
train_data.append(built_ner("I work for Autodesk.", "Autodesk", "ORG"))
train_data.append(built_ner("Model Derivative API provides translation", "Model Derivative API", "API"))
train_data.append(built_ner("The Model Derivative API used in conjunction with the Viewer", "Model Derivative API", "API"))
train_data.append(built_ner("I would like to automate Revit with the Design Automation API", "Design Automation API", "API"))

#Result
"""
[('I want to create a cloud-based service connected to Revit Server.',
  {'entities': [(52, 64, 'PRODUCT')]}),
 ("I'm new to the Forge API unsure where a design parameter is required",
  {'entities': [(15, 24, 'API')]}),
 ('Autodesk Forge is my Platform of choice',
  {'entities': [(0, 14, 'PRODUCT')]}),
 ("I've uploaded a Revit model to my OSS bucket.",
  {'entities': [(34, 37, 'SERVICE')]}),
 ('Changes are sent to a central BIM 360 server.',
  {'entities': [(30, 37, 'PRODUCT')]}),
 ('All of this is possible on the IFC format.',
  {'entities': [(31, 34, 'FORMAT')]}),
 ('The native file format for Revit is RVT.',
  {'entities': [(36, 39, 'FORMAT')]}),
 ('I work for Autodesk.', {'entities': [(11, 19, 'ORG')]}),
 ('Model Derivative API provides translation', {'entities': [(0, 20, 'API')]}),
 ('The Model Derivative API used in conjunction with the Viewer',
  {'entities': [(4, 24, 'API')]}),
 ('I would like to automate Revit with the Design Automation API',
  {'entities': [(40, 61, 'API')]})]
"""

##  Model

In [10]:
# Define our Variables
model = None
output_dir=Path(r"C:\Users\Mohamed Nabil\Documents\Project")
n_iter=100

# Load the Model
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

# Set up the pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


## Train the Recognizer

In [14]:
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*disable_pipes):
    optimizer = nlp.begin_training()
    
    for i in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        
        batches = minibatch(train_data , size = compounding(1.0,16.0,1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            
            example = []
            # Update the model with iterating each text
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            
                # Update the model
                nlp.update(
                           example,
                           drop=0.6,
                           sgd = optimizer,
                           losses=losses
                           )
            print(losses)



{'ner': 3.9130441546440125}
{'ner': 9.756462335586548}
{'ner': 18.387743711471558}
{'ner': 24.09941667318344}
{'ner': 28.787225484848022}
{'ner': 32.48712056875229}
{'ner': 37.16712445020676}
{'ner': 41.502202570438385}
{'ner': 46.66268026828766}
{'ner': 51.19598364830017}
{'ner': 56.567626893520355}
{'ner': 58.59824362397194}
{'ner': 62.82389858365059}
{'ner': 67.43411475419998}
{'ner': 71.34712636470795}
{'ner': 75.0627084672451}
{'ner': 78.57050827145576}
{'ner': 82.64747086167336}
{'ner': 86.53704449534416}
{'ner': 89.01103357970715}
{'ner': 92.80088892392814}
{'ner': 95.51865520747378}
{'ner': 97.98626872646855}
{'ner': 101.43128713435726}
{'ner': 103.78917792654829}
{'ner': 105.87483384396182}
{'ner': 108.50892024236964}
{'ner': 111.89089665637584}
{'ner': 115.83165249779995}
{'ner': 118.99355206072505}
{'ner': 121.41851892103296}
{'ner': 124.70147078678201}
{'ner': 128.2746282617627}
{'ner': 129.56310425479387}
{'ner': 132.08758445237254}
{'ner': 134.07728900121947}
{'ner': 137.

{'ner': 76.19694535715487}
{'ner': 78.91730988785964}
{'ner': 81.0210228038555}
{'ner': 82.78789484129045}
{'ner': 85.57133228390701}
{'ner': 87.74119335078656}
{'ner': 88.85373337185963}
{'ner': 91.97859326457281}
{'ner': 93.60176979457636}
{'ner': 98.33842998560893}
{'ner': 100.63750247421292}
{'ner': 102.13151063189896}
{'ner': 104.0559860992699}
{'ner': 105.43559793555058}
{'ner': 107.65777863500723}
{'ner': 109.89293929219009}
{'ner': 112.93896257671175}
{'ner': 114.31806002723631}
{'ner': 117.43998399333675}
{'ner': 119.59506794887882}
{'ner': 123.69482139786949}
{'ner': 125.21790205777832}
{'ner': 126.77782548111607}
{'ner': 129.1641594009448}
{'ner': 131.59455655319067}
{'ner': 133.08260355129056}
{'ner': 135.59583050318548}
{'ner': 137.69783335543258}
{'ner': 139.49023864957059}
{'ner': 141.9478730293086}
{'ner': 143.48336772218437}
{'ner': 144.93269501080187}
{'ner': 146.23059397855224}
{'ner': 147.6106728488424}
{'ner': 150.40262250164307}
{'ner': 151.58862215072514}
{'ner':

{'ner': 114.27210661491168}
{'ner': 116.30236363175935}
{'ner': 117.70574154644054}
{'ner': 119.06878424766528}
{'ner': 120.29308643803897}
{'ner': 122.31763412495403}
{'ner': 123.04596398803332}
{'ner': 126.71501519126977}
{'ner': 127.95369371959055}
{'ner': 131.76965044932277}
{'ner': 131.79477413440225}
{'ner': 132.4820636940775}
{'ner': 133.94423664360534}
{'ner': 134.99772927915205}
{'ner': 136.60369091031237}
{'ner': 142.14105992140836}
{'ner': 143.28132906165285}
{'ner': 145.41774067875542}
{'ner': 146.3553160126492}
{'ner': 147.33782043609375}
{'ner': 151.78203438983314}
{'ner': 154.47090902616787}
{'ner': 157.33881228958023}
{'ner': 158.71671457390457}
{'ner': 160.60137150804195}
{'ner': 161.77428297827402}
{'ner': 163.30554906587832}
{'ner': 0.5610376456538404}
{'ner': 2.362333032339407}
{'ner': 3.4986586072401877}
{'ner': 4.560384730643937}
{'ner': 5.796093467185647}
{'ner': 10.392995911466514}
{'ner': 11.106408033566632}
{'ner': 13.221801030445286}
{'ner': 16.69101417196191

{'ner': 6.375227412860332}
{'ner': 8.145104021771035}
{'ner': 8.610580680755229}
{'ner': 9.736732437686921}
{'ner': 10.338488691601906}
{'ner': 11.371745508353412}
{'ner': 12.91223734727062}
{'ner': 15.67129740147338}
{'ner': 17.577928672972863}
{'ner': 21.175348531855164}
{'ner': 22.07880683180623}
{'ner': 23.204527163994122}
{'ner': 24.77027620481361}
{'ner': 25.922569329865592}
{'ner': 27.202110260559923}
{'ner': 28.410218414782005}
{'ner': 30.609960406362685}
{'ner': 31.588840971599016}
{'ner': 35.61055179110027}
{'ner': 35.82164164602938}
{'ner': 37.49198630394916}
{'ner': 38.12359547573029}
{'ner': 39.11437464216999}
{'ner': 42.9145234171275}
{'ner': 43.2062832537904}
{'ner': 45.094582113265645}
{'ner': 45.44114042636875}
{'ner': 46.65217728764116}
{'ner': 47.422378621320554}
{'ner': 48.759297142279344}
{'ner': 50.41608432556319}
{'ner': 53.135278249660544}
{'ner': 53.65855207732368}
{'ner': 54.30655882706491}
{'ner': 56.56233381834283}
{'ner': 62.09720367857376}
{'ner': 62.35151

{'ner': 52.06199642454405}
{'ner': 52.13268945291124}
{'ner': 52.95404695818761}
{'ner': 54.70262732085678}
{'ner': 54.714546700780375}
{'ner': 56.40907778875131}
{'ner': 58.176051425737555}
{'ner': 60.53380896352724}
{'ner': 62.04212387774192}
{'ner': 62.1962619825852}
{'ner': 64.23041620204873}
{'ner': 65.5717254139278}
{'ner': 66.7031852757475}
{'ner': 67.67020912872856}
{'ner': 68.29711062356469}
{'ner': 69.31387175200848}
{'ner': 70.70271934861697}
{'ner': 71.01931850598872}
{'ner': 74.15482241372732}
{'ner': 75.47166113516506}
{'ner': 76.20168804045177}
{'ner': 77.29996232306304}
{'ner': 79.29732443785299}
{'ner': 80.68937891046053}
{'ner': 82.40699929682594}
{'ner': 82.45330643509594}
{'ner': 82.65439923162523}
{'ner': 82.9200046322694}
{'ner': 84.10791610551051}
{'ner': 85.87377235850856}
{'ner': 85.88439070132492}
{'ner': 87.90229034542685}
{'ner': 88.67140664786419}
{'ner': 89.0897985610838}
{'ner': 89.59549608225478}
{'ner': 89.6953101831154}
{'ner': 91.16662016016559}
{'ner

...

{'ner': 6.520004117931344e-08}
{'ner': 6.520031084901404e-08}
{'ner': 0.0037435991771182613}
{'ner': 0.3623274197713315}
{'ner': 0.36239249380437716}
{'ner': 0.36239249380437716}
{'ner': 0.3623981612927677}
{'ner': 0.3623981612927678}
{'ner': 0.3629214720877379}
{'ner': 0.3629215050690864}
{'ner': 0.3629215050690864}
{'ner': 0.3629234962076382}
{'ner': 0.3629234962076382}
{'ner': 0.3657369453425363}
{'ner': 0.3657369453425364}
{'ner': 0.3657369453425366}
{'ner': 0.36573695204285245}
{'ner': 0.3657369525304554}
{'ner': 0.4009042256653474}
{'ner': 0.4009042259674255}
{'ner': 2.365520418528423}
{'ner': 2.365520418528423}
{'ner': 2.365520418528423}
{'ner': 2.365520641554841}
{'ner': 9.66290013526822}
{'ner': 9.66290013526822}
{'ner': 9.662900135268279}
{'ner': 9.662900135268279}
{'ner': 9.662900135268302}
{'ner': 9.66290013526906}
{'ner': 9.662900135269156}
{'ner': 9.662900135269156}
{'ner': 9.662900135270917}
{'ner': 9.662900135270993}
{'ner': 9.662900135821731}
{'ner': 9.662900135821731}

{'ner': 5.3373088885707896}
{'ner': 5.337343104103642}
{'ner': 5.337343104103643}
{'ner': 5.337343104103656}
{'ner': 8.815903735458523}
{'ner': 8.815903735458523}
{'ner': 8.81590373546459}
{'ner': 8.8159037357307}
{'ner': 8.816051033738205}
{'ner': 8.816051033738262}
{'ner': 8.816338285547175}
{'ner': 8.816338285547175}
{'ner': 8.816338285547175}
{'ner': 8.817736334376463}
{'ner': 8.817736334376464}
{'ner': 8.817736334430839}
{'ner': 8.817740967713922}
{'ner': 8.817740967713922}
{'ner': 8.817740973825059}
{'ner': 8.8177409779683}
{'ner': 8.81774102014832}
{'ner': 8.81774102014832}
{'ner': 8.817741020152168}
{'ner': 8.817741020152173}
{'ner': 8.817741020152173}
{'ner': 8.817741020182792}
{'ner': 8.817741020182993}
{'ner': 8.817741020187174}
{'ner': 10.650606258376872}
{'ner': 12.650605781539713}
{'ner': 12.65060579451582}
{'ner': 12.65060579451582}
{'ner': 12.65060579451582}
{'ner': 12.65060579451582}
{'ner': 12.650606088211466}
{'ner': 12.656216408521109}
{'ner': 12.656216408521278}
{'

{'ner': 10.073288618128792}
{'ner': 10.073288618151368}
{'ner': 11.385757742945374}
{'ner': 11.385757742945374}
{'ner': 11.385757743047963}
{'ner': 11.385757743047963}
{'ner': 11.385758340663518}
{'ner': 11.385758340663518}
{'ner': 11.385758340663578}
{'ner': 11.385758340688664}
{'ner': 11.385758340688664}
{'ner': 11.385758349382666}
{'ner': 11.385758349382666}
{'ner': 12.854703732477159}
{'ner': 12.854703732477159}
{'ner': 14.850952812181443}
{'ner': 14.85574550188738}
{'ner': 14.85574550188739}
{'ner': 14.855745502346458}
{'ner': 14.855745547395518}
{'ner': 14.855745547407144}
{'ner': 7.681896363465057e-08}
{'ner': 7.681918443216604e-08}
{'ner': 7.681918443316221e-08}
{'ner': 9.03005674773746e-06}
{'ner': 9.030144607110015e-06}
{'ner': 9.030144636179481e-06}
{'ner': 9.056986992268259e-06}
{'ner': 1.1718816728012011}
{'ner': 1.1718816728057353}
{'ner': 1.1718816728057353}
{'ner': 1.1718816728057353}
{'ner': 4.393891093212197}
{'ner': 4.393891093212197}
{'ner': 4.393891093212197}
{'ner

{'ner': 6.474786764945447}
{'ner': 6.474786764945447}
{'ner': 6.47478676495918}
{'ner': 6.47478676495918}
{'ner': 6.474786764960544}
{'ner': 6.474786764960557}
{'ner': 6.532262153010636}
{'ner': 6.532262153010636}
{'ner': 6.532262153010636}
{'ner': 6.5322621530116844}
{'ner': 6.533967582811411}
{'ner': 6.533967654006686}
{'ner': 6.533967654006686}
{'ner': 8.637681017415728}
{'ner': 8.637681017415728}
{'ner': 8.637741466102414}
{'ner': 8.637741479970147}
{'ner': 8.637741479980285}
{'ner': 8.63774148004643}
{'ner': 8.637741482661793}
{'ner': 8.637741482661793}
{'ner': 8.637741511705844}
{'ner': 12.567425673845737}
{'ner': 12.56742567386247}
{'ner': 12.56742567433243}
{'ner': 16.265041059387972}
{'ner': 16.265041059388086}
{'ner': 16.265645134011056}
{'ner': 16.26617212638236}
{'ner': 18.191574530284072}
{'ner': 22.141764478371368}
{'ner': 22.141915891797105}
{'ner': 22.14191589179732}
{'ner': 22.14191589179747}
{'ner': 22.14191591030545}
{'ner': 22.141915910305457}
{'ner': 22.14191598727

{'ner': 9.065243392079744}
{'ner': 9.065243392079747}
{'ner': 9.065243392086233}
{'ner': 9.065247536032405}
{'ner': 9.065550919587475}
{'ner': 9.065550937302914}
{'ner': 9.065550937302914}
{'ner': 11.071182592029954}
{'ner': 11.071182592029954}
{'ner': 11.071559549971003}
{'ner': 11.071559550307237}
{'ner': 11.07155955030743}
{'ner': 11.4105451722366}
{'ner': 11.4105451722366}
{'ner': 11.555067034505047}
{'ner': 11.555067034505052}
{'ner': 11.555067338137016}
{'ner': 14.904697336344109}
{'ner': 16.901535190729483}
{'ner': 16.901535190729483}
{'ner': 16.901535190729483}
{'ner': 16.901535190730197}
{'ner': 20.89687684589019}
{'ner': 22.896711867276544}
{'ner': 22.89671188929487}
{'ner': 22.896711889294878}
{'ner': 22.896711889294878}
{'ner': 22.89671188930352}
{'ner': 22.89671188930352}
{'ner': 22.89671188930352}
{'ner': 24.893475483530324}
{'ner': 24.893475547893473}
{'ner': 24.893506079090997}
{'ner': 24.893506079090997}
{'ner': 24.89350608237954}
{'ner': 24.893506082577343}
{'ner': 26

## Save The Model

In [15]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)   

Saved model to C:\Users\Mohamed Nabil\Documents\Project


In [16]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in train_data[:10]:
    doc = nlp(text)
    print('Text :', text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Loading from C:\Users\Mohamed Nabil\Documents\Project
Text : play music for justin bieber
Entities [('justin bieber', 'artist')]
Text : turn on the kitchen light
Entities [('turn on', 'light_on'), ('kitchen', 'room_type')]
Text : ask wikipedia about computer
Entities [('computer', 'topic')]
Text : how is the weather next week in zagazig
Entities [('next week', 'date'), ('zagazig', 'location')]
Text : tell me the latest news in science
Entities [('science', 'news_topic')]
Text : can you play shallow song
Entities [('shallow', 'song_name')]
Text : i want to hear is jazz
Entities [('jazz', 'music_genre')]
Text : can you play benkhaf song
Entities [('benkhaf', 'song_name')]
Text : shuffle songs by coldplay
Entities [('coldplay', 'artist')]
Text : set an alarm for five am
Entities [('five am', 'time')]


----------------------------------------------------------------------------------------------------------------